# Explaination about this Notebook
In this notebook I gathered all the news links related to the DAX40 from the finanzen.net website.

In the notebook `get_news.ipynb` I gathered the news from those links.

In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import selenium as se
from time import sleep
from tqdm import tqdm as tq

# Driver for Firefox, Chrome, Edge, etc.
from selenium import webdriver

# Mode of locating html elements: ID, CSS_SELECTOR, XPATH, ...
from selenium.webdriver.common.by import By               

# Using specific keyboard keys like ENTER, ESCAPE, ...
from selenium.webdriver.common.keys import Keys

# Methods for dropdown
from selenium.webdriver.support.select import Select

import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException


In [2]:
url = r'https://www.finanzen.net/index/dax/marktberichte'

In [3]:
# Use the Firefox browser to open the URL
driver = webdriver.Firefox()
driver.get(url)

# Set the time period for the driver to wait for the element to appear
driver.implicitly_wait(10)

def pop_up_window(driver, css_button, css_window):
    try:
        # Attempt to find and click the element
        element = driver.find_element(By.CSS_SELECTOR, css_button)
        element.click()
    except:
        #print("Element not found - Try to search in iframes")

        try:
            # Accept the Datenschutz
            iframes = driver.find_elements(By.CSS_SELECTOR, css_window)
            #print(f"Number of iframes: {len(iframes)}")

            for i in range(len(iframes)):
                try:
                    driver.switch_to.frame(iframes[i])
                    #print(f"Switched to Pop-Up {i}")
                    
                    # Attempt to find and click the element
                    element = driver.find_element(By.CSS_SELECTOR, css_button)
                    element.click()
                    
                    #print("Pop-Up found and clicked")
                    break  # Exit the loop once the element is found and clicked
                except NoSuchElementException:
                    #print(f"Element not found in Pop-Up {i}")
                    pass
                except ElementNotInteractableException:
                    # print(f"Element not interactable in Pop-Up {i}")
                    pass
                except Exception as e:
                    print(f"An error occurred in Pop-Up {i}: {str(e)}")
                finally:
                    # Switch back to the main content before checking the next iframe
                    driver.switch_to.default_content()
        except Exception as e:
            print(f'Could not find the Pop-Up button: {str(e)}')

    return driver


In [4]:
driver = pop_up_window(driver, css_button=".button-responsive-primary", css_window="iframe")

In [5]:
# look for the highest page in the pagination
css_selector = 'li.pagination__item:nth-child(11) > a:nth-child(1)'
max_pages = driver.find_element(By.CSS_SELECTOR, css_selector)
print(max_pages.text)

425


In [6]:
def get_news_links(driver, url):
    all_links = pd.DataFrame(columns=['links'])
    for page in tq(range(1, int(max_pages.text)), desc='Pages'):

        driver = pop_up_window(driver, css_button=".finWebpushCloseButton", css_window="#finWebpushNotificationModal")

        article_list = []
        url_page =url + f'?p={page}'
        driver.get(url_page)
        #driver.implicitly_wait(10)
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser') 
        # find article
        
        articles = soup.find_all('article')
        for article in articles:
            article_soup = BeautifulSoup(str(article), 'html.parser')
            links = article_soup.find_all('a', href=True)
            for link in links:
                # print(link)
                # print(link['href'])
                article_list.append(link['href'])
                
        # print(f'Page {page} - {len(article_list)} articles found')
        # drop hrefs wich start with https://
        article_list = [x for x in article_list if not x.startswith('https://')]
        article_list = pd.DataFrame(article_list, columns=['links'])
        all_links = pd.concat([all_links, article_list], ignore_index=True)

        # safty first
        if page % 10 == 0: # save every 10 pages
            dir_name = 'links'
            all_links.to_csv(f'..\\data\\links\\links-{page}.csv', index=False)
            pass

        # if page == 3:
        #     break
            
    return all_links

In [7]:
article_list = get_news_links(driver=driver, url=url)
# This code was running for 2:30 hours

Pages: 100%|██████████| 424/424 [2:35:57<00:00, 22.07s/it]  


In [8]:
len(article_list)


40290

In [12]:
# check for duplicates in article_list
df_temp = article_list.copy()
df_temp.drop_duplicates(inplace=True)
# drop link if it contains '?p='
df_temp = df_temp[~df_temp['links'].str.contains('\?p=')]
df_temp.reset_index(drop=True, inplace=True)
df_temp.to_csv(r'..\data\links\article_list.tsv', index=False, sep='\t')
article_list = df_temp['links'].tolist()
print(f'len(article_list): {len(article_list)}')

AttributeError: 'list' object has no attribute 'drop_duplicates'

To continue reading the code please look at `get_news.iypnb`!

---
---